In [83]:
import numpy as np
import matplotlib.pyplot as plt
import keras

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

## Import del dataset
from keras.datasets import mnist

In [84]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [53]:
## Funzione di stampa delle immagini del dataset per provarlo
# Non utile al progetto ma come test si
def plot_input_img(i):
    plt.imshow(X_train[i], cmap='binary')
    plt.title(y_train[i])
    plt.show()

In [ ]:
for i in range(10):
    plot_input_img(i)

In [85]:
## Pre processing delle immagini
# Normalizzazione delle immagini in floating point nell'intervallo [0,1]
X_train = X_train.astype(np.float32)/255
X_test = X_test.astype(np.float32)/255

# REshape della dimensione delle immagini, aggiungo un asse per ottenere (28,28,1)
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

# Trasformo le etichette delle classi in una rappresentazione 'One hot encoding'
# Assegno un vettore binario unico per ogni classe, dove un elemento del vettore è 1 (rappresenta la classe corretta) e gli altri sono 0
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)


In [86]:
X_train.shape

(60000, 28, 28, 1)

In [87]:
model = Sequential()

# Aggiunta primo strato di convoluzione e max pooling
model.add(Conv2D(32, (3,3), input_shape = (28,28,1), activation = 'relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

# Aggiunta secondo strato di convoluzione, questa volta con 64 filtri 3x3 + altro max pooling
model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2, 2)))

# Strato di flatten per convertire dati bidimensionali in vettoriale piano
model.add(Flatten())

# Aggiunta dropout con prob a 25% per prevenire overfitting
model.add(Dropout(0.25))

model.add(Dense(10, activation = 'softmax'))

In [88]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 1600)              0         
                                                                 
 dropout_3 (Dropout)         (None, 1600)             

In [89]:
# Compilazione modello
model.compile(optimizer = 'adam', loss = keras.losses.categorical_crossentropy, metrics = ['accuracy'])

In [90]:
## Aggiunta Callbacks
# Funzioni di monitoraggio che possono intervenire durante il training del modello
# Earlystopping - guarda che il valore ddella metrica 'val_acc' dell'epoche precedente non scenda sotto il delta
es = EarlyStopping(monitor = 'val_accuracy', min_delta = 0.01, patience = 4, verbose = 1 )

# Model checkpoint -salva il modello ogni volta che viene ottenuta un accuratezza migliore di quella precedente
mc = ModelCheckpoint("./bestAcc_model.h5", monitor = "val_accuracy", verbose = 1, save_best_only = True)

cb = [es, mc]

# Model Training

In [91]:
## Model training
his = model.fit(X_train, y_train, epochs = 25, validation_split = 0.3, callbacks = cb)

Epoch 1/25
1312/1313 [============================>.] - ETA: 0s - loss: 0.2079 - accuracy: 0.9379
Epoch 1: val_accuracy improved from -inf to 0.97450, saving model to .\bestAcc_model.h5
1313/1313 [==============================] - 26s 19ms/step - loss: 0.2079 - accuracy: 0.9379 - val_loss: 0.0821 - val_accuracy: 0.9745
Epoch 2/25
   6/1313 [..............................] - ETA: 29s - loss: 0.1012 - accuracy: 0.9688

c:\Users\denicola\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1311/1313 [============================>.] - ETA: 0s - loss: 0.0728 - accuracy: 0.9776
Epoch 2: val_accuracy improved from 0.97450 to 0.98267, saving model to .\bestAcc_model.h5
1313/1313 [==============================] - 25s 19ms/step - loss: 0.0727 - accuracy: 0.9776 - val_loss: 0.0557 - val_accuracy: 0.9827
Epoch 3/25
1312/1313 [============================>.] - ETA: 0s - loss: 0.0547 - accuracy: 0.9826
Epoch 3: val_accuracy improved from 0.98267 to 0.98406, saving model to .\bestAcc_model.h5
1313/1313 [==============================] - 26s 20ms/step - loss: 0.0547 - accuracy: 0.9826 - val_loss: 0.0550 - val_accuracy: 0.9841
Epoch 4/25
1309/1313 [============================>.] - ETA: 0s - loss: 0.0442 - accuracy: 0.9862
Epoch 4: val_accuracy improved from 0.98406 to 0.98528, saving model to .\bestAcc_model.h5
1313/1313 [==============================] - 26s 20ms/step - loss: 0.0442 - accuracy: 0.9863 - val_loss: 0.0503 - val_accuracy: 0.9853
Epoch 5/25
1312/1313 [=================

In [92]:
model_S = keras.models.load_model("C://Users//denicola//Documents//Handwritten Digit Recognition//bestAcc_model.h5")

In [93]:
score = model_S.evaluate(X_test, y_test)
print(f"Model accuracy: {score[1]}")

313/313 [==============================] - 1s 4ms/step - loss: 0.0297 - accuracy: 0.9897
Model accuracy: 0.9897000193595886
